In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# 모형 결합

모형 결합(model combining) 방법은 앙상블 방법론(ensemble methods)이라고도 한다. 이는 특정한 하나의 예측 방법이 아니라 복수의 예측 모형을 결합하여 더 나은 성능의 예측을 하려는 시도이다.
모형 결합 방법을 사용하면 일반적으로 계산량은 증가하지만 다음과 같은 효과가 있다.
단일 모형을 사용할 때 보다 성능 분산이 감소하고, 즉 과최적화를 방지한다.
개별 모형이 성능이 안좋을 경우에는 결합 모형의 성능이 더 향상된다.
모형 결합 방법은 크게 나누어 취합(aggregation) 방법론과 부스팅(boosting) 방법론으로 나눌 수 있다.

취합 방법론은 사용할 모형의 집합이 이미 결정되어 있지만
부스팅 방법론은 사용할 모형을 점진적으로 늘려간다.
각 방법론의 대표적인 방법들은 아래와 같다.

취합 방법론

다수결 (Majority Voting)

배깅 (Bagging)

랜덤포레스트 (Random Forests)

부스팅 방법론

에이다부스트 (AdaBoost)

그레디언트 부스트 (Gradient Boost)

여기에서는 우선 취합 방법에 대해 살펴본다.



In [ ]:
## 유방암 진단 데이터셋 구축
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
print(cancer.DESCR)

# 데이터 프레임으로 만들기
cnc_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
sy = pd.Series(cancer.target, dtype="category")
sy = sy.cat.rename_categories(cancer.target_names)
cnc_df['class'] = sy
cnc_df.tail()

# train / test set
from sklearn.utils import shuffle
cnc_df_shu=shuffle(cnc_df)
train=cnc_df_shu.iloc[:int(len(cnc_df)*0.7),:]
test=cnc_df_shu.iloc[int(len(train)):,:]
train_X=train.iloc[:,:-1]
train_Y=train.iloc[:,-1]
test_X=test.iloc[:,:-1]
test_Y=test.iloc[:,-1]

## 다수결 방법¶
다수결 방법은 가장 단순한 모형 결합 방법으로 전혀 다른 모형도 결합할 수 있다. 다수결 방법은 Hard Voting 과 Soft Voting 두 가지로 나뉘어진다.

hard voting: 단순 투표. 개별 모형의 결과 기준

soft voting: 가중치 투표. 개별 모형의 조건부 확률의 합 기준

Scikit-Learn 의 ensemble 서브패키지는 다수결 방법을 위한 VotingClassifier 클래스를 제공한다. 입력 인수는 다음과 같다.

- estimators: 개별 모형 목록, 리스트나 named parameter 형식으로 입력

- voting: 문자열 {hard, soft} hard voting 과 soft voting 선택. 디폴트는 hard

- weights: 사용자 가중치 리스트

\
\
***동일한 문제를 세가지 다른 방법으로 풀어본다. ***

- 로지스틱 회귀 모형

- QDA 모형

- 가우시안 나이브베이즈 모형

마지막으로 세가지 모형을 다수결로 합친 모형을 VotingClassifier 클래스로 만들었다. 다만 세가지 모형의 가중치가 각각 1, 1, 2로 가우시안 나이브베이즈 모형의 가중치를 높였다.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier

## 로지스틱 모형
model_logit=LogisticRegression()

## 가우시안 베이즈 모형
model_nb=GaussianNB()

# 판별분석
model_qda=QuadraticDiscriminantAnalysis()

## 3개 모형 투표 : 앙상블
ensemble=VotingClassifier(estimators=[("lr",model_logit),("nb",model_nb),("qda",model_qda)],voting='soft')
probs=[c.fit(train_X,train_Y).predict_proba(test_X) for c in (model_logit,model_nb,model_qda,ensemble)]
class1_1=[pr[0,0] for pr in probs]
class2_1=[pr[0,1] for pr in probs]

# 시각화


ind = np.arange(4)
width = 0.35  # bar width
p1 = plt.bar(ind, np.hstack(([class1_1[:-1], [0]])), width, color='green')
p2 = plt.bar(ind + width, np.hstack(([class2_1[:-1], [0]])), width, color='lightgreen')
p3 = plt.bar(ind, [0, 0, 0, class1_1[-1]], width, color='blue')
p4 = plt.bar(ind + width, [0, 0, 0, class2_1[-1]], width, color='steelblue')

plt.xticks(ind + 0.5 * width, ['Logistic', 'QDA', 'Gaussian NB', 'Soft Ensemble'])
plt.ylim([0, 1.1])
plt.title("Each Results of 3 Models' ")
plt.legend([p1[0], p2[0]], ['class 1', 'class 2'], loc='upper left')
plt.show()

## 배깅
앞서 모형 결합에서 사용하는 독립적인 모형의 수가 많을 수록 성능 향상이 일어날 가능성이 높다는 것을 알았다. 각각 다른 확률 모형을 사용하는데에는 한계가 있으므로 보통은 배깅 방법을 사용하여 같은 확률 모형을 쓰지만 서로 다른 결과를 출력하는 다수의 모형을 만든다.

배깅(bagging)은 동일한 모형과 모형 모수를 사용하는 대신 부트스트래핑(bootstrapping)과 유사하게 트레이닝 데이터를 랜덤하게 선택해서 다수결 모형을 적용한다.

트레이닝 데이터를 선택하는 방법에 따라 다음과 같이 부르기도 한다.

같은 데이터 샘플을 중복사용(replacement)하지 않으면: Pasting

같은 데이터 샘플을 중복사용(replacement)하면 Bagging

데이터가 아니라 다차원 독립 변수 중 일부 차원을 선택하는 경우에는: Random Subspaces

데이터 샘플과 독립 변수 차원 모두 일부만 랜덤하게 사용하면: Random Patches

성능 평가시에는 트레이닝용으로 선택한 데이터가 아닌 다른 데이터를 사용할 수도 있다. 이런 데이터를 OOB(out-of-bag) 데이터라고 한다.

Scikit-Learn 의 ensemble 서브패키지는 배깅 모형 결합을 위한 BaggingClassifier 클래스를 제공한다. 사용법은 다음과 같다.

- base_estimator: 기본 모형

- n_estimators: 모형 갯수. 디폴트 10

- bootstrap: 데이터의 중복 사용 여부. 디폴트 True

- max_samples: 데이터 샘플 중 선택할 샘플의 수 혹은 비율. 디폴트 1.0

- bootstrap_features: 특징 차원의 중복 사용 여부. 디폴트 False

- max_features: 다차원 독립 변수 중 선택할 차원의 수 혹은 비율 1.0

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

# 데이터셋 마련
iris=load_iris()
X,Y=iris.data[:,[0,2]],iris.target
iris_df=pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df["class"]=pd.DataFrame(data=iris.target)


# train / test set
iris_df_shu=shuffle(iris_df)
train=iris_df_shu.iloc[:int(len(iris_df)*0.7),:]
test=iris_df_shu.iloc[int(len(train)):,:]
train_X=train.iloc[:,:-1]
train_Y=train.iloc[:,-1]
test_X=test.iloc[:,:-1]
test_Y=test.iloc[:,-1]

# 의사결정나무, bagging 의사결정나무
model1=DecisionTreeClassifier(max_depth=10,random_state=0).fit(train_X,train_Y)
model2=BaggingClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=100,random_state=0).fit(train_X,train_Y)

# 예측
print(model1.predict(test_X)==test_Y)
print(model2.predict(test_X)==test_Y)

# accuracy 측정
from sklearn.model_selection import cross_val_score
print(round(cross_val_score(model1, test_X, test_Y, scoring="accuracy", cv=5).mean(),2),
    "\n",round(cross_val_score(model2, test_X, test_Y, scoring="accuracy", cv=5).mean(),2))

## 랜덤포레스트
랜덤포레스트(Random Forest)는 의사 결정 나무(Decision Tree)를 개별 모형으로 사용하는 모형 결합 방법을 말한다.

랜덤포레스트는 데이터 특징차원의 일부만 선택하여 사용한다. 하지만 노드 분리시 모든 독립 변수들을 비교하여 최선의 독립 변수를 선택하는 것이 아니라 독립 변수 차원을 랜덤하게 감소시킨 다음 그 중에서 독립 변수를 선택한다. 이렇게 하면 개별 모형들 사이의 상관관계가 줄어들기 때문에 모형 성능의 변동이 감소하는 효과가 있다.

이러한 방법을 극단적으로 적용한 것이 Extremely Randomized Trees 모형으로 이 경우에는 각 노드에서 랜덤하게 독립 변수를 선택한다.

랜덤포레스트와 Extremely Randomized Trees 모형은 각각 RandomForestClassifier클래스와 ExtraTreesClassifier 클래스로 구현되어 있다.

랜덤포레스트의 장점 중 하나는 각 독립 변수의 중요도(feature importance)를 계산할 수 있다는 점이다.

포레스트 안에서 사용된 모든 노드에 대해 어떤 독립 변수를 사용하였고 그 노드에서 얻은 information gain을 구할 수 있으므로 각각의 독립 변수들이 얻어낸 information gain의 평균을 비교하면 어떤 독립 변수가 중요한지를 비교할 수 있다.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
## Kfold 교차검증
# 데이터 셋
X=cancer.data
Y=cancer.target

# 모델 생성
## 로지스틱 모형
model_logit=LogisticRegression()

## 가우시안 베이즈 모형
model_nb=GaussianNB()

# 판별분석
model_qda=QuadraticDiscriminantAnalysis()

## 3개 모형 투표 : 앙상블
ensemble=VotingClassifier(estimators=[("lr",model_logit),("nb",model_nb),("qda",model_qda)],voting='soft',weights=[1,1.5,1])
cv=KFold(5,shuffle=True,random_state=0)
acc=cross_val_score(ensemble,X,Y,scoring="accuracy",cv=cv)
print("acc.mean : ",round(acc.mean(),3),"\n","acc.std : ",round(acc.std(),3))

In [ ]:
model1 = DecisionTreeClassifier(max_depth=5)
model2 = BaggingClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100)
accuracy1 = cross_val_score(model1, X, Y, scoring="accuracy", cv=cv)
accuracy2 = cross_val_score(model2, X, Y, scoring="accuracy", cv=cv)
accuracy1.mean(), accuracy1.std(), accuracy2.mean(), accuracy2.std()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

forest = ExtraTreesClassifier(n_estimators=500, random_state=0).fit(X, Y)
accuracy = cross_val_score(forest, X, Y, scoring="accuracy", cv=cv)
accuracy.mean(), accuracy.std()

In [ ]:
idx = np.argsort(forest.feature_importances_)
names = cancer.feature_names[idx]
values = forest.feature_importances_[idx]

plt.figure(figsize=(10, 10))
plt.barh(names, values)
plt.title("Breast Cancer Feature Importance")
plt.show()